# Bab 16: Natural Language Processing with RNNs, Attention, and Transformers

### 1. Pendahuluan

Bab 16 melanjutkan pembahasan dari bab sebelumnya, dengan fokus menerapkan Jaringan Saraf Tiruan (JST) untuk tugas **Natural Language Processing (NLP)**. NLP adalah salah satu bidang *Deep Learning* yang paling menarik dan menantang, mencakup tugas-tugas seperti penerjemahan mesin, analisis sentimen, dan pembuatan teks.

Bab ini akan membahas:
* **Representasi Teks:** Cara mengubah teks menjadi vektor numerik yang dapat diproses oleh model, termasuk teknik *word embeddings*.
* **Arsitektur Encoder-Decoder:** Pola arsitektur yang umum digunakan untuk tugas *sequence-to-sequence* seperti penerjemahan mesin.
* **Attention Mechanism:** Sebuah inovasi kunci yang memungkinkan model untuk fokus pada bagian-bagian paling relevan dari input saat menghasilkan output, mengatasi keterbatasan RNN konvensional.
* **Arsitektur Transformer:** Arsitektur modern yang revolusioner yang sepenuhnya mengandalkan mekanisme *attention*, tanpa menggunakan RNN sama sekali.

---

### 2. Representasi Teks dan Word Embeddings

Kata-kata harus dikonversi menjadi representasi numerik.
* **One-Hot Encoding:** Sederhana tetapi tidak efisien karena menghasilkan vektor yang sangat besar dan *sparse*, serta tidak menangkap kemiripan antar kata.
* **Word Embeddings:** Representasi vektor padat dan berdimensi rendah yang menangkap makna semantik kata. Kata-kata yang mirip secara makna akan memiliki vektor yang berdekatan. *Embedding* ini dapat dipelajari dari awal atau menggunakan model *pre-trained* seperti Word2Vec atau GloVe.

Di Keras, `keras.layers.Embedding` digunakan untuk mengubah indeks integer kata menjadi *embedding vector*.

---

### 3. Arsitektur Encoder-Decoder untuk Penerjemahan Mesin

Untuk tugas *sequence-to-sequence* (seperti penerjemahan dari satu bahasa ke bahasa lain), arsitektur **Encoder-Decoder** sangat umum digunakan.
1.  **Encoder:** Sebuah RNN (misalnya, LSTM atau GRU) yang memproses urutan input dan meringkasnya menjadi satu vektor konteks (biasanya *hidden state* terakhir dari RNN).
2.  **Decoder:** Sebuah RNN lain yang mengambil vektor konteks dari *encoder* dan menghasilkan urutan output langkah demi langkah. Pada setiap langkah, *decoder* menerima kata yang dihasilkan sebelumnya sebagai input untuk menghasilkan kata berikutnya.

#### a. Teacher Forcing
Selama pelatihan, input ke *decoder* pada setiap langkah adalah kata target yang sebenarnya dari langkah sebelumnya, bukan kata yang diprediksi oleh *decoder* itu sendiri. Teknik ini disebut *teacher forcing* dan membantu menstabilkan serta mempercepat pelatihan.


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Contoh kerangka Encoder-Decoder
# (Ini adalah contoh konseptual, implementasi penuh memerlukan data dan preprocessing)
vocab_size = 10000
embed_size = 128

# Encoder
encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
encoder_embeddings = keras.layers.Embedding(vocab_size, embed_size)(encoder_inputs)
_, encoder_state_h, encoder_state_c = keras.layers.LSTM(
    128, return_state=True)(encoder_embeddings)
encoder_state = [encoder_state_h, encoder_state_c]

# Decoder
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_embeddings = keras.layers.Embedding(vocab_size, embed_size)(decoder_inputs)
decoder_lstm = keras.layers.LSTM(128, return_sequences=True)
decoder_outputs = decoder_lstm(decoder_embeddings, initial_state=encoder_state)
decoder_dense = keras.layers.Dense(vocab_size, activation="softmax")
output = decoder_dense(decoder_outputs)

# Membuat model
model_encoder_decoder = keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[output])

---

### 4. Attention Mechanisms
Kelemahan utama dari arsitektur Encoder-Decoder dasar adalah semua informasi dari urutan input harus dipadatkan menjadi satu vektor konteks berukuran tetap. Ini menjadi bottleneck untuk kalimat yang panjang.

Attention Mechanism mengatasi ini dengan memungkinkan decoder untuk "melihat" dan fokus pada bagian-bagian yang relevan dari seluruh urutan input pada setiap langkah saat menghasilkan output.

Cara Kerja: Pada setiap langkah decoding, mekanisme attention menghitung skor keselarasan (alignment score) antara hidden state decoder saat ini dan setiap hidden state encoder. Skor ini kemudian diubah menjadi bobot melalui fungsi softmax. Context vector untuk langkah tersebut adalah jumlah terbobot dari semua hidden state encoder.

Mekanisme ini terbukti sangat efektif dan menjadi dasar bagi arsitektur Transformer.

---

### 5. Arsitektur Transformer
Arsitektur Transformer, yang diperkenalkan dalam paper "Attention Is All You Need", sepenuhnya meninggalkan RNN dan hanya mengandalkan mekanisme attention. Ini memungkinkan paralelisasi yang masif dan telah menjadi arsitektur standar untuk banyak tugas NLP.

# a. Arsitektur Transformer
* *Positional Embeddings*: Karena tidak ada rekurensi, Transformer menambahkan positional embeddings ke input embeddings untuk memberikan informasi tentang posisi kata dalam urutan.
* *Multi-Head Attention*: Alih-alih melakukan satu fungsi attention, multi-head attention memproyeksikan input ke beberapa subspace yang berbeda dan menerapkan attention secara paralel. Ini memungkinkan model untuk fokus pada   berbagai aspek dari urutan input.
* *Encoder*: Terdiri dari tumpukan blok identik. Setiap blok memiliki lapisan Multi-Head Attention diikuti oleh lapisan Feed-Forward Network. Terdapat koneksi residual (residual connections) dan normalisasi lapisan (layer normalization) di sekitar setiap sub-lapisan.
* *Decoder*: Mirip dengan encoder, tetapi memiliki lapisan Multi-Head Attention kedua yang melakukan attention pada output dari encoder. Lapisan attention pertamanya di-masking untuk mencegahnya "melihat" kata-kata di masa depan (causal attention).

In [ ]:
# (Keras menyediakan layer `keras.layers.MultiHeadAttention` siap pakai)

class PositionalEncoding(keras.layers.Layer):
    # Implementasi untuk menambahkan informasi posisi
    pass

class MultiHeadAttention(keras.layers.Layer):
    def __init__(self, n_heads, causal=False, **kwargs):
        super().__init__(**kwargs)
        self.n_heads = n_heads
        self.causal = causal
        # Lapisan-lapisan ini akan dibuat di dalam metode build()
        # self.q_dense, self.k_dense, self.v_dense, self.attention, self.out_dense

    def build(self, batch_input_shape):
      # Implementasi build
      pass
      
    def call(self, inputs, mask=None):
      # Logika call
      pass

# Contoh kerangka model Transformer Encoder
embed_size = 128
max_steps = 500
vocab_size = 10000

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
embeddings = keras.layers.Embedding(vocab_size, embed_size)(encoder_inputs)
encoder_in = PositionalEncoding(max_steps, max_dims=embed_size)(embeddings)

# Z = encoder_in
# for N in range(6): # Tumpukan 6 blok encoder
#    Z = MultiHeadAttention(...)(Z) # Blok attention
#    Z = FeedForward(...)(Z)      # Blok feed-forward

---

### 6. Kesimpulan

Bab ini menunjukkan bagaimana NLP telah berevolusi dari RNN sederhana ke arsitektur yang lebih canggih seperti LSTM, GRU, dan puncaknya, Transformer. Konsep word embeddings memungkinkan representasi teks yang kaya. Arsitektur Encoder-Decoder dengan mekanisme attention mengatasi keterbatasan RNN untuk urutan panjang, sementara Transformer, dengan sepenuhnya mengandalkan attention, telah menetapkan standar baru dalam kinerja dan efisiensi pelatihan untuk berbagai tugas NLP.